# Create An Adversarial patch for ResNet34

The foundations of this code were taken from the in class notebook 'adversarial_attacks.ipynb'. I adapted the functions to fit the needs of my idea. If i had more time I would like to add some testing and i would adapt the place_patch function to make it more robust in real world environments. It seems like the label i chose is slightly challenging so pperformance is not guarunteed.

**Install & import required packages**

In [27]:
pip install torch torchvision numpy matplotlib tqdm requests

1477.17s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


This code was adapted from the in class notebook.

In [30]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn as nn
import torch.utils.data as data
from torchvision import transforms
from PIL import Image
import requests
import zipfile
import io

# import model

In [23]:
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
model = torchvision.models.resnet34(weights='IMAGENET1K_V1')
model = model.to(device)


# Import Imagenet data

In [15]:
response = requests.get("https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial10/TinyImageNet.zip")
print(response)

if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        zip_ref.extractall("/content/")

<Response [200]>


You may need to manually load the wales flag file into the Colab files. 

In [16]:
transform = transforms.Compose([
        transforms.Resize(64),   # Resize the image to match the patch size
        transforms.ToTensor(),           # Convert the image to a tensor 
    ])

    # Load the image
seed_image = Image.open('Flag_of_Wales.svg.png').convert("RGB")  

seed_image_tensor = transform(seed_image)

In [17]:
# Mean and Std from ImageNet
NORM_MEAN = np.array([0.485, 0.456, 0.406])
NORM_STD = np.array([0.229, 0.224, 0.225])
# No resizing and center crop necessary as images are already preprocessed.
plain_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN,
                         std=NORM_STD)
])
dataset = torchvision.datasets.ImageFolder(root='TinyImageNet', transform=plain_transforms)

In [18]:
def place_patch(img, patch):
    """place patch function from class notebook"""
    for i in range(img.shape[0]):
        h_offset = np.random.randint(0,img.shape[2]-patch.shape[1]-1)
        w_offset = np.random.randint(0,img.shape[3]-patch.shape[2]-1)
        img[i,:,h_offset:h_offset+patch.shape[1],w_offset:w_offset+patch.shape[2]] = patch_forward(patch)
    return img

In [19]:
TENSOR_MEANS, TENSOR_STD = torch.FloatTensor(NORM_MEAN)[:,None,None], torch.FloatTensor(NORM_STD)[:,None,None]
def patch_forward(patch):
    # Map patch values from [-infty,infty] to ImageNet min and max
    patch = (torch.tanh(patch) + 1 - 2 * TENSOR_MEANS) / (2 * TENSOR_STD)
    return patch

In [20]:
def patch_attack(model, target_class, patch_size=64, num_epochs=5):
    """training the patch. Takes a model, target class and patch size"""

    train_loader = data.DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True, num_workers=8)

    # Create parameter and optimizer
    if not isinstance(patch_size, tuple):
        patch_size = (patch_size, patch_size)
    patch = nn.Parameter(seed_image_tensor, requires_grad=True) #initialise from seed image
    optimizer = torch.optim.SGD([patch], lr=1e-1, momentum=0.8)
    loss_module = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(num_epochs):
        t = tqdm(train_loader, leave=False)
        for img, _ in t:
            img = place_patch(img, patch)
            img = img.to(device)
            pred = model(img)
            labels = torch.zeros(img.shape[0], device=pred.device, dtype=torch.long).fill_(target_class)
            loss = loss_module(pred, labels)
            optimizer.zero_grad()
            loss.mean().backward()
            optimizer.step()
            t.set_description(f"Epoch {epoch}, Loss: {loss.item():4.2f}")

    return patch.data

In [21]:
def visualise_patch(patch):
    """turn patch into viewable object"""
    patch = patch.detach().cpu().permute(1, 2, 0)
    patch = patch.numpy()
    plt.imshow(patch)
    plt.axis('off')
    plt.savefig("GO_WALES.png", bbox_inches='tight', pad_inches=0)
    plt.show()

In [29]:
patch = patch_attack(model, 911) #911 refers to the wooden spoon, In the six nations rugby competition the team that comes last is said to have won the 'wooden spoon', Wales were the winners of this in 2024
visualise_patch(patch)


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
